In [5]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
METADATA_CSV_PATH = "F:\\Deepfake-Audio-Detector\\datasets\\for-dataset\\generated_metadata.csv"
OUTPUT_DATASET_DIR = "F:\\Deepfake-Audio-Detector\\datasets\\new_for_dataset"

try:
    df = pd.read_csv(METADATA_CSV_PATH)
    print(f"Đã tải metadata từ: {METADATA_CSV_PATH}")
    print(f"Tổng số bản ghi: {len(df)}")
except FileNotFoundError:
    print(
        f"Lỗi: Không tìm thấy file metadata CSV tại {METADATA_CSV_PATH}. Vui lòng kiểm tra đường dẫn."
    )
    exit()

Đã tải metadata từ: F:\Deepfake-Audio-Detector\datasets\for-dataset\generated_metadata.csv
Tổng số bản ghi: 123481


In [7]:
required_columns = ["utterance_id", "label", "domain", "file_path"]
if not all(col in df.columns for col in required_columns):
    print(f"Lỗi: File metadata CSV phải chứa các cột sau: {required_columns}")
    exit()

df["label"] = df["label"].astype(str).str.lower()
df = df[df["label"].isin(["real", "fake"])]
if df.empty:
    print(
        "Lỗi: Không có bản ghi hợp lệ nào với nhãn 'real' hoặc 'fake' sau khi làm sạch."
    )
    exit()


In [8]:
unique_ids = df["utterance_id"].unique()
print(f"Tổng số utterance_id duy nhất: {len(unique_ids)}")

train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
train_ids, val_ids = train_test_split(train_ids, test_size=0.125, random_state=42)  

print(f"Số lượng utterance_id trong tập train: {len(train_ids)}")
print(f"Số lượng utterance_id trong tập val: {len(val_ids)}")
print(f"Số lượng utterance_id trong tập test: {len(test_ids)}")

def assign_split(row):
    if row["utterance_id"] in test_ids:
        return "test"
    elif row["utterance_id"] in val_ids:
        return "val"
    else:
        return "train"


df["split"] = df.apply(assign_split, axis=1)

Tổng số utterance_id duy nhất: 47730
Số lượng utterance_id trong tập train: 33411
Số lượng utterance_id trong tập val: 4773
Số lượng utterance_id trong tập test: 9546


In [9]:
# --- Bước 3: Tăng cường số lượng sample (không rò rỉ) ---
processed_df_list = []
for split_type in ["train", "val", "test"]:
    split_df = df[df["split"] == split_type].copy()
    if split_type in ["val", "test"]:
        split_df["domain_priority"] = split_df["domain"].apply(
            lambda x: 0 if x == "original" else 1
        )
        
        split_df = split_df.sort_values(
            by=["utterance_id", "domain_priority"]
        ).drop_duplicates(subset=["utterance_id"], keep="first")
        split_df = split_df.drop(columns=["domain_priority"])
    processed_df_list.append(split_df)

final_df = pd.concat(processed_df_list).reset_index(drop=True)
print(f"Tổng số bản ghi sau khi xử lý domain: {len(final_df)}")

Tổng số bản ghi sau khi xử lý domain: 100755


In [10]:
# --- Bước 4: Tạo cấu trúc thư mục chuẩn và di chuyển file ---
# Tạo thư mục gốc dataset nếu chưa tồn tại
os.makedirs(OUTPUT_DATASET_DIR, exist_ok=True)

# Tạo các thư mục con
for split_type in ["train", "val", "test"]:
    for label_type in ["fake", "real"]:
        path = os.path.join(OUTPUT_DATASET_DIR, split_type, label_type)
        os.makedirs(path, exist_ok=True)
        print(f"Đã tạo thư mục: {path}")

# Di chuyển các file âm thanh gốc
count_moved = 0
count_skipped = 0
for index, row in final_df.iterrows():
    # Lấy đường dẫn file âm thanh gốc từ cột 'file_path'
    source_audio_file_path = row["file_path"]
    label = row["label"]
    split = row["split"]

    # Lấy tên file gốc (bao gồm cả các hậu tố .wav_16k.wav_norm.wav...)
    original_filename = os.path.basename(source_audio_file_path)

    # Đường dẫn đích trong cấu trúc dataset mới
    destination_audio_file_path = os.path.join(
        OUTPUT_DATASET_DIR, split, label, original_filename
    )

    if os.path.exists(source_audio_file_path):
        try:
            # Sử dụng shutil.copy để sao chép file
            shutil.copy(
                source_audio_file_path, destination_audio_file_path
            )  # Hoặc shutil.move nếu bạn muốn di chuyển hẳn
            count_moved += 1
        except Exception as e:
            print(
                f"Lỗi khi di chuyển/sao chép file {source_audio_file_path} đến {destination_audio_file_path}: {e}"
            )
    else:
        print(
            f"Cảnh báo: Không tìm thấy file âm thanh gốc tại {source_audio_file_path}. Bỏ qua."
        )
        count_skipped += 1

print("Quá trình hoàn tất.")
print(f"Tổng số file âm thanh đã di chuyển/sao chép: {count_moved}")
print(f"Tổng số file âm thanh bị bỏ qua (không tìm thấy): {count_skipped}")
print(f"Cấu trúc dataset đã được tạo tại: {OUTPUT_DATASET_DIR}")

Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\train\fake
Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\train\real
Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\val\fake
Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\val\real
Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\test\fake
Đã tạo thư mục: F:\Deepfake-Audio-Detector\datasets\test\real
Quá trình hoàn tất.
Tổng số file âm thanh đã di chuyển/sao chép: 100755
Tổng số file âm thanh bị bỏ qua (không tìm thấy): 0
Cấu trúc dataset đã được tạo tại: F:\Deepfake-Audio-Detector\datasets


In [ ]:
# old_path = "F:\\Deepfake-Audio-Detector\\datasets\\for-dataset"
# new_path = "F:\\Deepfake-Audio-Detector\\datasets\\old_for_dataset"
# if os.path.exists(new_path):
#     print(f"Thư mục '{new_path}' đã tồn tại.")
# else:
#     os.rename(old_path, new_path)
#     print(f"Đã đổi tên '{old_path}' thành '{new_path}'.")

In [ ]:
# set_types = ["train", "val", "test"]
# base_dir = "F:\\Deepfake-Audio-Detector\\datasets"
# base_destination = "F:\\Deepfake-Audio-Detector\\datasets\\for-dataset"
# os.makedirs(base_destination, exist_ok=True)

# for set_type in set_types:
#     base_dir_path = os.path.join(base_dir, set_type)
#     if os.path.exists(base_dir_path):
        
#         shutil.move(base_dir_path, base_destination)
#         print(f"Đã di chuyển thư mục {base_dir_path} đến for-dataset.")
#     else:
#         print(f"Thư mục {base_dir_path} không tồn tại, không cần di chuyển.")